In [1]:
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(ShinyCell))

In [272]:
spatial_in_tissue.obj <- readRDS("../../reports/customers/Rai_merged/spatial_in_tissue.obj2.rds")


spatial_in_tissue.obj

[[1]]
An object of class Seurat 
24919 features across 2052 samples within 1 assay 
Active assay: D00717 (24919 features, 0 variable features)
 1 image present: D00717_spatial

[[2]]
An object of class Seurat 
24919 features across 1894 samples within 1 assay 
Active assay: D00719 (24919 features, 0 variable features)
 1 image present: D00719_spatial

[[3]]
An object of class Seurat 
24919 features across 2495 samples within 1 assay 
Active assay: D00739 (24919 features, 0 variable features)
 1 image present: D00739_spatial

[[4]]
An object of class Seurat 
24919 features across 2039 samples within 1 assay 
Active assay: D00740 (24919 features, 0 variable features)
 1 image present: D00740_spatial

[[5]]
An object of class Seurat 
24919 features across 2441 samples within 1 assay 
Active assay: D00899 (24919 features, 0 variable features)
 1 image present: D00899_spatial

[[6]]
An object of class Seurat 
24919 features across 1183 samples within 1 assay 
Active assay: D00900 (24919 fea

# Storing spatial maps as dimensional reduction

In [23]:
samples <- list()

for (i in c(1:length(spatial_in_tissue.obj))) {
    samples[[i]] <- names((spatial_in_tissue.obj[[i]])@assays)
    }

samples <- unlist(samples)
samples

[1] "D00717" "D00719" "D00739" "D00740" "D00899" "D00900"

In [379]:
# find null columns
toRemove <- list()
for (i in seq_along(samples)) {
toRemove[[i]] <- names(which(colSums(is.na(spatial_in_tissue.obj[[i]]@assays[[1]]@counts))>0))
    }
toRemove <- unlist(toRemove)
head(toRemove)

[1] "D00717#CGCATACAAGAGTCAA-1" "D00717#AAACATCGACGCTCGA-1"
[3] "D00717#CACTTCGAACACAGAA-1" "D00717#ACACAGAACTGTAGCC-1"
[5] "D00717#ACATTGGCCATACCAA-1" "D00717#ATGCCTAAGAATCTGA-1"

In [ ]:
# prepare spatial maps
# columns should come first and rows need to be negative to see similar maps in shiny app

In [321]:
Spatial_df <- list()
image_name <-  list()
for (i in seq_along(samples)) {
        
Spatial_df[[i]] <- as.data.frame(spatial_in_tissue.obj[[i]]@images[[1]]@coordinates[,c(5,4)])
colnames(Spatial_df[[i]]) <- paste0("Spatial_", 1:2)
Spatial_df[[i]]$Spatial_2 <- -(Spatial_df[[i]]$Spatial_2)
}

# Spatial_df

In [132]:
Spatial_all <- data.table()
for (i in seq_along(samples)) {

tmp <- as.data.frame(rbind(Spatial_df[[i]]))
Spatial_all <- rbindlist(list(Spatial_all, tmp))
    }
rownames(Spatial_all)<- unlist(lapply(Spatial_df, function(x) {rownames(x)}))
Spatial_all <- as.data.frame.data.frame(Spatial_all)
Spatial_all <-Spatial_all[which(!rownames(Spatial_all)%in%toRemove),]


In [224]:
Spatial_df_all <- list()

for (i in samples){

df <- Spatial_all
# needle <- i
    
tmp1 <- df[grepl(paste0("^", i), row.names(df)),]
tmp2 <- df[!grepl(paste0("^", i), row.names(df)),]
tmp2[,] <- 0
tmp <- rbind(tmp1,tmp2)
Spatial_df_all[[i]] <- tmp
    }

In [230]:
# get gene scores
# merge seurat objects


In [248]:
temp <- list()
for (i in seq_along(samples)) {
temp[[i]] <- as.data.frame(spatial_in_tissue.obj[[i]]@assays[[1]]@counts)
temp[[i]]$cells<- rownames(temp[[i]])
rownames(temp[[i]])<-NULL
    }


In [265]:
combined_mat <-  Reduce(function(...) merge(..., all = TRUE, by="cells"), temp)
combined_mat[is.na(combined_mat)]<- 0
rownames(combined_mat)<- combined_mat$cells
combined_mat$cells<-NULL
combined_mat<-combined_mat[,which(colnames(combined_mat)%in%rownames(Spatial_all))]
combined_mat <- as.matrix(combined_mat)


In [266]:
# meta data

In [274]:
# chamge some column names
for (i in seq_along(samples)) {
colnames(spatial_in_tissue.obj[[i]]@meta.data) <- gsub(samples[i],""
                                                            ,colnames(spatial_in_tissue.obj[[i]]@meta.data))
    }


In [284]:
List_md <- list()
for (i in seq_along(samples)) {
    List_md[[i]] <- spatial_in_tissue.obj[[i]]@meta.data
    }
meta.data<- do.call("rbind",List_md)[colnames(combined_mat),]


In [286]:
# creat merged seurat object
combined <- CreateSeuratObject(
  counts = combined_mat,
  assay = "scATAC",
  meta.data = meta.data
)
combined

An object of class Seurat 
24919 features across 11943 samples within 1 assay 
Active assay: scATAC (24919 features, 0 variable features)

In [287]:
combined@meta.data$Clusters <- factor(combined@meta.data$Clusters,levels = c(
"C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13","C14","C15","C16","C17","C18","C19","C20"
))

In [314]:
for (i in samples) {
    for (j in seq_along(samples)){
key = paste0(i,"Spatial_")
combined[[key]] <- CreateDimReducObject(embeddings = as.matrix(Spatial_df_all[[j]])
                                              , key = key
                                              , assay = DefaultAssay(combined))
}
    }

In [316]:

combined@reductions

$D00717Spatial_
A dimensional reduction object with key D00717Spatial_ 
 Number of dimensions: 2 
 Projected dimensional reduction calculated:  FALSE 
 Jackstraw run: FALSE 
 Computed using assay: scATAC 

$D00719Spatial_
A dimensional reduction object with key D00719Spatial_ 
 Number of dimensions: 2 
 Projected dimensional reduction calculated:  FALSE 
 Jackstraw run: FALSE 
 Computed using assay: scATAC 

$D00739Spatial_
A dimensional reduction object with key D00739Spatial_ 
 Number of dimensions: 2 
 Projected dimensional reduction calculated:  FALSE 
 Jackstraw run: FALSE 
 Computed using assay: scATAC 

$D00740Spatial_
A dimensional reduction object with key D00740Spatial_ 
 Number of dimensions: 2 
 Projected dimensional reduction calculated:  FALSE 
 Jackstraw run: FALSE 
 Computed using assay: scATAC 

$D00899Spatial_
A dimensional reduction object with key D00899Spatial_ 
 Number of dimensions: 2 
 Projected dimensional reduction calculated:  FALSE 
 Jackstraw run: FALSE 
 C

In [342]:
xlim <- list()
for (j in seq_along(samples)){
    xlim[[j]] <- c(min(Spatial_df[[j]][,1]),max(Spatial_df[[j]][,1]))
    }
xlim <- unlist(xlim)
min(xlim)
max(xlim)

ylim <- list()
for (j in seq_along(samples)){
    ylim[[j]] <- c(min(Spatial_df[[j]][,2]),max(Spatial_df[[j]][,2]))
    }
ylim <- unlist(ylim)
min(ylim)
max(ylim)

[1] 137.459

[1] 3644.153

[1] -3622.754

[1] -206.4297

In [ ]:
# we also need to run Variable Features

In [343]:
combined <- NormalizeData(combined, normalization.method = "LogNormalize", scale.factor = 1000)
combined <- FindVariableFeatures(combined, selection.method = "vst", nfeatures = 200)


In [ ]:
# add UMAP!

In [344]:
UMAPHarmony <- read.csv("../../reports/customers/Rai_merged/UMAPHarmony.csv")
UMAPHarmony<-UMAPHarmony[match(colnames(combined),UMAPHarmony$X),]

rownames(UMAPHarmony) <- UMAPHarmony$X
UMAPHarmony$X <- NULL
UMAPHarmony <- as.matrix(UMAPHarmony)
combined[["UMAPHarmony"]] <- CreateDimReducObject(embeddings = UMAPHarmony
                                              , key = "UMAPHarmony_"
                                              , assay = DefaultAssay(combined))




# ADD Motif Deviation scores as new object

In [ ]:
# the same steps in the above would be repeated for motifs

In [346]:
motif_object <- readRDS("../../reports/customers/Rai_merged/motif_object.rds")


# motif_object
# find null columns & filter them out
toRemove <- list()
for (i in seq_along(samples)) {
toRemove[[i]] <- names(which(colSums(is.na(motif_object[[i]]@assays[[1]]@counts))>0))
    }
toRemove <- unlist(toRemove)


# prepare spatial maps
# columns should come first and rows need to be negative to see similar maps in shiny app
Spatial_df <- list()
image_name <-  list()
for (i in seq_along(samples)) {
        
Spatial_df[[i]] <- as.data.frame(motif_object[[i]]@images[[1]]@coordinates[,c(5,4)])
colnames(Spatial_df[[i]]) <- paste0("Spatial_", 1:2)
Spatial_df[[i]]$Spatial_2 <- -(Spatial_df[[i]]$Spatial_2)
}

Spatial_all <- data.table()
for (i in seq_along(samples)) {

tmp <- as.data.frame(rbind(Spatial_df[[i]]))
Spatial_all <- rbindlist(list(Spatial_all, tmp))
    }
rownames(Spatial_all)<- unlist(lapply(Spatial_df, function(x) {rownames(x)}))
Spatial_all <- as.data.frame.data.frame(Spatial_all)
Spatial_all <-Spatial_all[which(!rownames(Spatial_all)%in%toRemove),]

Spatial_df_all <- list()

for (i in samples){

df <- Spatial_all
# needle <- i
    
tmp1 <- df[grepl(paste0("^", i), row.names(df)),]
tmp2 <- df[!grepl(paste0("^", i), row.names(df)),]
tmp2[,] <- 0
tmp <- rbind(tmp1,tmp2)
Spatial_df_all[[i]] <- tmp
    }

# get gene scores
# merge seurat objects
temp <- list()
for (i in seq_along(samples)) {
temp[[i]] <- as.data.frame(motif_object[[i]]@assays[[1]]@counts)
temp[[i]]$cells<- rownames(temp[[i]])
rownames(temp[[i]])<-NULL
    }
combined_mat <-  Reduce(function(...) merge(..., all = TRUE, by="cells"), temp)
combined_mat[is.na(combined_mat)]<- 0
rownames(combined_mat)<- combined_mat$cells
combined_mat$cells<-NULL
combined_mat<-combined_mat[,which(colnames(combined_mat)%in%rownames(Spatial_all))]
combined_mat <- as.matrix(combined_mat)

# meta data
# chamge some column names
for (i in seq_along(samples)) {
colnames(motif_object[[i]]@meta.data) <- gsub(samples[i],"",colnames(motif_object[[i]]@meta.data))}

List_md <- list()
for (i in seq_along(samples)) {
    List_md[[i]] <- motif_object[[i]]@meta.data
    }
meta.data_motif<- do.call("rbind",List_md)[colnames(combined_mat),]
                        
# creat merged seurat object
combined_motif <- CreateSeuratObject(
  counts = combined_mat,
  assay = "scATAC",
  meta.data = meta.data_motif
)
                        
combined_motif@meta.data$Clusters <- factor(combined_motif@meta.data$Clusters,levels = c(
"C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13","C14","C15","C16","C17","C18","C19","C20"))
                        
for (i in samples) {
    for (j in seq_along(samples)){
key = paste0(i,"Spatial_")
combined_motif[[key]] <- CreateDimReducObject(embeddings = as.matrix(Spatial_df_all[[j]])
                                              , key = key
                                              , assay = DefaultAssay(combined_motif))
}
    }   

combined_motif <- NormalizeData(combined_motif, normalization.method = "LogNormalize", scale.factor = 1000)
combined_motif <- FindVariableFeatures(combined_motif, selection.method = "vst", nfeatures = 200)
                        
UMAPHarmony <- read.csv("../../reports/customers/Rai_merged/UMAPHarmony.csv")
UMAPHarmony<-UMAPHarmony[match(colnames(combined_motif),UMAPHarmony$X),]

rownames(UMAPHarmony) <- UMAPHarmony$X
UMAPHarmony$X <- NULL
UMAPHarmony <- as.matrix(UMAPHarmony)

combined_motif[["UMAPHarmony"]] <- CreateDimReducObject(embeddings = UMAPHarmony
                                              , key = "UMAPHarmony_"
                                              , assay = DefaultAssay(combined_motif))



# Create ShinyCell configuration and configure settings¶

In [ ]:
# make shiny app files

In [ ]:

scConf1 = createConfig(combined)
makeShinyFiles(seu, scConf1, gex.assay = "scATAC", gex.slot = "data",
               gene.mapping = TRUE, shiny.prefix = "sc1",
               shiny.dir = "shinyAppMulti/",
               default.gene1 = "CD8A", default.gene2 = "DCT",
               default.multigene = c("DCT","PDCD1","CD19","TIGIT","CD8A","SEMA4D","GZMA","CCL3","CD4","SDF4","B3GALT6"),
               default.dimred = c("UMAPHarmony_1", "UMAPHarmony_2"))

scConf2 = createConfig(combined_motif)
makeShinyFiles(seu, scConf2, gex.assay = "scATAC", gex.slot = "counts",
               gene.mapping = TRUE, shiny.prefix = "sc2",
               shiny.dir = "shinyAppMulti/",
               default.gene1 = "HOXD8_537", default.gene2 = "TCF3_21",
               default.multigene = c(
                  'ZNF148-222','KLF5-175','ZBTB7A-258','KLF4-208','MAZ-178','ZFY-166','KLF15-223','SP3-247'),
               default.dimred = c("UMAPHarmony_1", "UMAPHarmony_2"))

In [ ]:
citation = list(
  title   = "Rai Lab Data Analysis"
                       )
makeShinyCodesMulti(
  shiny.title = "Rai Lab Data Analysis", 
    shiny.footnotes = citation,
  shiny.prefix = c("sc1", "sc2"),
  shiny.headers = c("Gene Accessibility", "Peak/Motifs"), 
  shiny.dir = "shinyAppMulti/") 

# Required data after creation shiny objects

In [ ]:
sc1def <- readRDS("shinyAppMulti/sc1def.rds")

# add xlim and ylim
sc1def$xlim1 <- min(xlim)
sc1def$xlim2 <- max(xlim)

sc1def$ylim1 <- min(ylim)
sc1def$ylim2 <- max(ylim)

# add more default 

sc1def$meta1<- "Clusters"
sc1def$meta2<- "treatment"

# different used treatment names
sc1def$treatment1<- sort(unique(combined@meta.data$treatment))[1]
sc1def$treatment2<- sort(unique(combined@meta.data$treatment))[2]